### Loading the data

In [3]:
import os
print(os.getcwd())

/Users/keegz_dsouza/Documents/Code/FINAL/Voice-Cloning


In [1]:
from data_scripts.data_loading import load_data  # New import from our streamlined loader

dataset, loader = load_data(
    processed_root="data/processed_data",  # update path as needed
    speakers_per_batch=40,       # number of speakers per batch
    utterances_per_speaker=10,   # number of utterances per speaker
    num_workers=0
)

In [2]:
# Check the shape of the first batch only
for batch in loader:
    print(batch.data.shape)
    break

(400, 160, 40)
